In [1]:
def ROEA_afliqe(archivo, estacion, plot=False, xlim=None):
    """Extrae las series del Anuario de Aforos del CEDEX del embalse indicado
    
    Entrada:
    --------
    archivo:  str. Ruta y nombre del archivo con los datos de los embalses del ROEA
    estacion: int. Código de la estacion ROEA del embalse
    plot:     boolean. Si se quiere mostrar la gráfica de las series
    xlim:     list. Fechas de inicio y fin de la gráfica
    
    Salida:
    -------
    data:     DataFrame. Serie diaria de caudal de entrada, salida (m³/s) y reserva (hm³) en el embalse.
    """
    
    # SERIE
    # -----
    # cargar datos
    data = pd.read_csv(archivo, parse_dates=True)
    # seleccionar datos de la estación
    data = data.loc[data['ref_ceh'] == estacion, :]
    # crear índice de fecha
    data.index = pd.to_datetime(data.fecha, dayfirst=True)
    # calcular caudal de entrada
    data['entrada'] = np.nan
    for day1 in data.index[:-1]:
        day2 = day1 + datetime.timedelta(days=1)
        AV = data.loc[day2, 'reserva'] - data.loc[day1, 'reserva'] # hm³
        AV /= 0.0864 # m³/s
        if data.loc[day1, 'tipo'] == 1:
            data.loc[day2, 'entrada'] = AV + data.loc[day2, 'salida']
        elif data.loc[day1, 'tipo'] == 2:
            data.loc[day1, 'entrada'] = AV + data.loc[day1, 'salida']
    # redondear
    data.entrada = data.entrada.round(2)
    # eliminar columnas innecesarias
    data.drop(['fecha', 'ref_ceh', 'tipo'], axis=1, inplace=True)
    
    # GRÁFICA
    # -------
    if plot == True:
        fig, ax1 = plt.subplots(figsize=(15, 4))
        # caudal
        ax1.tick_params(labelsize=11)
        ax1.plot(data.entrada, 'steelblue', lw=.5, label='entrada')
        ax1.plot(data.salida, 'indianred', lw=.5, label='salida')
        qmax = np.ceil(data[['entrada', 'salida']].max().max() / 100) * 100
        ylim1 = (-.2 * qmax, qmax)
        if xlim is None:
            ax1.set(xlim=(xlim), ylim=ylim1)
        else:
            ax1.set(xlim(data.index[0], data.index[-1]), ylim=ylim1)
        yticks1 = np.arange(-.2 * qmax, qmax + 1, qmax / 5)
        ax1.set_yticks = yticks1
        ax1.set_yticklabels = yticks1
        ax1.set_ylabel('Q (m³/s)', fontsize=13)
        # volumen
        ax2 = ax1.twinx()
        ax2.tick_params(labelsize=11)
        ax2.fill_between(data.index, data.reserva, color='grey', alpha=.25, label='reserva')
        vmax = np.ceil(data.reserva.max() / 50) * 50
        ylim2 = (-.2 * vmax, vmax)
        if xlim is None:
            ax2.set(xlim=(xlim), ylim=ylim2)
        else:
            ax2.set(xlim(data.index[0], data.index[-1]), ylim=ylim2)
        yticks2 = np.arange(-.2 * vmax, vmax + 1, vmax / 5)
        ax2.set_yticks = yticks2
        ax2.set_yticklabels = yticks2
        ax2.set_ylabel('V (hm³)', fontsize=13)
        fig.legend(loc=8, ncol=3, fontsize=13);

    return data

In [3]:
def ROEA_afliq(archivo, estacion, plot=False, xlim=None):
    """Extrae las series del Anuario de Aforos del CEDEX de la estaicón de aforo indicada
    
    Entrada:
    --------
    archivo:  str. Ruta y nombre del archivo con los datos de los embalses del ROEA
    estacion: int. Código de la estacion de aforo ROEA
    
    Salida:
    -------
    data:     DataFrame. Serie diaria de caudal (m³/s) y nivel (m)
    """
    
    # cargar datos
    data = pd.read_csv(archivo, parse_dates=True)
    # seleccionar datos de la estación
    data = data.loc[data['indroea'] == estacion, :]
    # crear índice de fecha
    idx = [datetime.datetime.strptime(day.split()[0], '%m/%d/%Y') for day in data.fecha]
    data.index = idx
    data.index.name = 'fecha'
    data.drop(['indroea', 'fecha'], axis=1, inplace=True)
    
    if plot == True:
        plt.figure(figsize=(15, 4))
        plt.tick_params(labelsize=11)
        plt.plot(data.caudal, 'steelblue', linewidth=.5, label='entrada')
        plt.ylabel('Q (m³/s)', fontsize=13)
        plt.ylim(0)
        if xlim != None:
            plt.xlim(xlim)
        else:
            plt.xlim(data.index[0], data.index[-1])

    return data

In [ ]:
def RMSE_poly(x, y, coefs):
    """Función que calcula la raíz del error medio cuadrático.
    
    Entradas:
    ---------
    x:       Series. Valores observados de la variable 'x' (nivel)
    y:       Series. Valores observados de la variable 'y' (caudal)
    coefs:   list. Coeficientes de la regresión polinómica de 2º grado"""
    
    # serie interpolada
    yhat = coefs[0] * x**2 + coefs[1] * x + coefs[2]
    # Raíz del error cuadrático medio
    rmse = np.sqrt(np.mean((y - yhat)**2))
    
    return rmse

In [ ]:
def ratingCurve(x, a, b):
    """Función polinómica de segundo grado con ordenada en el origen igual a 0.
    
    Entradas:
    ---------
    x:       Series. Serie de valores de la variable x (Nivel)
    a:       float. Coeficiente del término de segundo grado
    b:       float. Coeficiente del término de primer grado"""
    
    return a * x**2 + b * x

In [ ]:
def plotRatingCurve(stage, discharge, params=(0, 1), Nmax=3, Qmax=100):
    """Gráfica con los datos observados de nivel y caudal, y la curva de gasto ajustada
    
    Entradas:
    ---------
    stage:   Series. Serie temporal de nivel
    discharge: Series. Serie temporal de caudal
    params:    list. Parámetros a y b de la curva de gasto ajustada. Si es 'None', no se dibuja la curva de gasto
    Nmax:      float. Valor máximo del eje x (nivel) en la gráfica
    Qmax:      float. Valor máximo del eje y (caudal) en la gráfica
    """
    
    # gráfico de dispersión
    plt.figure(figsize=(5, 5))

    # diagrama de dispersión de las observaciones
    plt.scatter(stage, discharge, s=2, label='observaciones')
    
    # curva de gasto
    if params is not None:
        x = np.linspace(0, Nmax, 50)
        y = ratingCurve(x, params[0], params[1])
        plt.plot(x, y, linewidth=1.2, c='dimgray', linestyle='-', label='ajuste')

    plt.xlim(0, Nmax)
    plt.ylim(0, Qmax)
    plt.xlabel('N (m)', fontsize=13)
    plt.ylabel('Q (m³/s)', fontsize=13)
    plt.legend(fontsize=11);